### 1. Resumen JOINs

Supongamos que tenemos dos tablas: `A` (izquierda) y `B` (derecha).

- `inner_join(A, B, by = ...)`  → **Solo coincidencias** (intersección).
- `left_join(A, B, by = ...)`   → Todas las filas de **A**, agrega columnas de B si hay match.
- `right_join(A, B, by = ...)`  → Todas las filas de **B**.
- `full_join(A, B, by = ...)`   → Todas las filas de A y B.
- `semi_join(A, B, by = ...)`   → Filas de A que **sí tienen match** en B (no agrega columnas de B).
- `anti_join(A, B, by = ...)`   → Filas de A que **NO tienen match** en B.

En la práctica, el más usado para enriquecer datos es **`left_join()`**.


### 2. Ejemplo clase anterior

- `alumnos`: información básica de alumnos.
- `notas`: notas finales de algunos alumnos.


In [ ]:
alumnos <- data.frame(
  id_alumno = 1:6,
  nombre = c("Ana", "Benjamín", "Carla", "Diego", "Elena", "Fernando"),
  curso = c("A", "B", "A", "C", "B", "A")
)

notas <- data.frame(
  id = c(2, 3, 4, 6, 7),
  nota_final = c(5.8, 6.5, 4.9, 5.3, 6.0)
)

alumnos
notas


id_alumno,nombre,curso
<int>,<chr>,<chr>
1,Ana,A
2,Benjamín,B
3,Carla,A
4,Diego,C
5,Elena,B
6,Fernando,A


id,nota_final
<dbl>,<dbl>
2,5.8
3,6.5
4,4.9
6,5.3
7,6.0


### 2.1 `inner_join()`
**Solo alumnos que tienen nota registrada.**


In [ ]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
alumnos %>%
  inner_join(notas, by = c("id_alumno" = "id"))



id_alumno,nombre,curso,nota_final
<dbl>,<chr>,<chr>,<dbl>
2,Benjamín,B,5.8
3,Carla,A,6.5
4,Diego,C,4.9
6,Fernando,A,5.3


### 2.2 `left_join()`
**Todos los alumnos**, y la nota si existe.


In [ ]:
alumnos %>%
  left_join(notas, by = c("id_alumno" = "id"))

id_alumno,nombre,curso,nota_final
<dbl>,<chr>,<chr>,<dbl>
1,Ana,A,NA
2,Benjamín,B,5.8
3,Carla,A,6.5
4,Diego,C,4.9
5,Elena,B,NA
6,Fernando,A,5.3


### 2.3 `right_join()`
**Todas las notas**, aunque el alumno no esté en la tabla de alumnos.


In [ ]:
alumnos %>%
  right_join(notas, by = c("id_alumno" = "id"))

id_alumno,nombre,curso,nota_final
<dbl>,<chr>,<chr>,<dbl>
2,Benjamín,B,5.8
3,Carla,A,6.5
4,Diego,C,4.9
6,Fernando,A,5.3
7,NA,NA,6.0


### 2.4 `full_join()`
Une **todos los alumnos y todas las notas**.


In [ ]:
alumnos %>%
  full_join(notas, by = c("id_alumno" = "id"))

id_alumno,nombre,curso,nota_final
<dbl>,<chr>,<chr>,<dbl>
1,Ana,A,NA
2,Benjamín,B,5.8
3,Carla,A,6.5
4,Diego,C,4.9
5,Elena,B,NA
6,Fernando,A,5.3
7,NA,NA,6.0


### 2.5 `semi_join()`
Alumnos que **sí tienen nota**, pero sin agregar la columna `nota_final`.


In [ ]:
alumnos %>%
  semi_join(notas, by = c("id_alumno" = "id"))

id_alumno,nombre,curso
<int>,<chr>,<chr>
2,Benjamín,B
3,Carla,A
4,Diego,C
6,Fernando,A


In [ ]:
alumnos %>%
  left_join(notas, by = c("id_alumno" = "id")) %>%
    filter(!is.na(nota_final)) %>%
    select(-nota_final)


id_alumno,nombre,curso
<dbl>,<chr>,<chr>
2,Benjamín,B
3,Carla,A
4,Diego,C
6,Fernando,A


### 2.6 `anti_join()`
Alumnos que **NO tienen nota**.


In [ ]:
alumnos %>%
  anti_join(notas, by = c("id_alumno" = "id"))

id_alumno,nombre,curso
<int>,<chr>,<chr>
1,Ana,A
5,Elena,B


In [ ]:
alumnos %>%
  left_join(notas, by = c("id_alumno" = "id")) %>%
    filter(is.na(nota_final)) %>%
    select(-nota_final)


id_alumno,nombre,curso
<dbl>,<chr>,<chr>
1,Ana,A
5,Elena,B


## 3. nycflights13
Se tiene la hipótesis de que el clima puede ser un factor importante en los retrasos, para ello cargue el dataset llamado "weather" del package "nycflights13"

**a)** Genere un dataset llamado *vuelos_ret_ampliado* que incluya tanto la información de los vuelos como del clima. (el match debe hacerse por "year","month","day","origin","time_hour").

In [ ]:
vuelos <- read.csv("flights.csv")
weather <- read.csv("weather.csv")

In [ ]:
vuelos %>% head()

weather %>% head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>
1,2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2,2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
3,2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
4,2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
5,2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
6,2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00


,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,EWR,2013,1,1,0,37.04,21.92,53.97,230,10.35702,11.91865,0,1013.9,10,2012-12-31 21:00:00
2,EWR,2013,1,1,1,37.04,21.92,53.97,230,13.80936,15.89154,0,1013.0,10,2012-12-31 22:00:00
3,EWR,2013,1,1,2,37.94,21.92,52.09,230,12.65858,14.56724,0,1012.6,10,2012-12-31 23:00:00
4,EWR,2013,1,1,3,37.94,23.00,54.51,230,13.80936,15.89154,0,1012.7,10,2013-01-01 00:00:00
5,EWR,2013,1,1,4,37.94,24.08,57.04,240,14.96014,17.21583,0,1012.8,10,2013-01-01 01:00:00
6,EWR,2013,1,1,6,39.02,26.06,59.37,270,10.35702,11.91865,0,1012.0,10,2013-01-01 03:00:00


In [ ]:
vuelos_ret_ampliado <- vuelos %>%
  left_join(weather, by = c("year","month","day","origin","time_hour"))


#### Retrasos 15 min

Cree una columna de **tipo int** llamada *arr_delay_alto*, que tome el valor 1 si el retraso de llegada es mayor o igual a 15 min y 0 en caso contrario. De ahora en adelante diremos que esta columna indica si existió un _retraso alto_ o no.

In [ ]:
#ifelse(condicion, valor si verdadero, valor si falso)
#case_when(condiciones, valor si verdadero)
# >= 15 & <= 20 -> 1
# > 20 & <= 30 -> 2
# -> 0


vuelos_ret_ampliado <- vuelos_ret_ampliado %>%
  mutate(arr_delay_alto = ifelse(arr_delay >= 15, 1, 0),
         arr_delay_alto_v2 = case_when( arr_delay >= 15 ~ 1,
                                       T  ~ 0 )   )




In [ ]:
# >= 15 & <= 20 -> 1
# > 20 & <= 30 -> 2
# -> 0

vuelos_ret_ampliado %>%
  mutate(arr_delay_alto = ifelse(arr_delay >= 15 & arr_delay <= 20, 1, ifelse(arr_delay > 20 & arr_delay <= 30, 2, 0)),
         arr_delay_alto_v2 = case_when( arr_delay >= 15 & arr_delay <= 20 ~ 1,
                                        arr_delay > 20 & arr_delay <= 30 ~ 2,
                                        T  ~ 0 )   ) %>%
                                          head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,⋯,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,arr_delay_alto,arr_delay_alto_v2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2013,1,1,517,515,2,830,819,11,UA,⋯,28.04,64.43,240,11.50780,13.24295,0,1012.5,10,0,0
2,2013,1,1,533,529,4,850,830,20,UA,⋯,26.06,54.97,260,16.11092,18.54012,0,1012.0,10,1,1
3,2013,1,1,542,540,2,923,850,33,AA,⋯,26.96,59.50,260,14.96014,17.21583,0,1012.7,10,0,0
4,2013,1,1,544,545,-1,1004,1022,-18,B6,⋯,26.96,59.50,260,14.96014,17.21583,0,1012.7,10,0,0
5,2013,1,1,554,600,-6,812,837,-25,DL,⋯,26.06,54.97,260,17.26170,19.86442,0,1011.8,10,0,0
6,2013,1,1,554,558,-4,740,728,12,UA,⋯,28.04,64.43,240,11.50780,13.24295,0,1012.5,10,0,0


#### Clima según retraso alto
    
**a)** ¿Cuál es la visibilidad (visib) promedio por tipo de retraso (alto y no alto)?

**b)** ¿Cuál es el valor de la precipitacion (precip) media por tipo de retraso (alto y no alto)?

**c)** ¿Cuál es el valor promedio de wind_gust (vientos fuertes)  por tipo de retraso?

**d)** ¿Qué se observa a partir del resumen anterior?

In [ ]:
#a
vuelos_ret_ampliado %>%
  group_by(arr_delay_alto)%>%
  summarise(promedio_visib = mean(visib, na.rm = T),
            prom_prec = mean(precip, na.rm = T)) %>%
    ungroup()

arr_delay_alto,promedio_visib,prom_prec
<dbl>,<dbl>,<dbl>
0,9.388411,0.001655788
1,8.683571,0.005283987
NA,7.696329,0.008103880


In [ ]:
#